In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query

from datetime import datetime

종목코드열람

In [2]:
shcode_list = query.query_shcode_list_today()

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1
            
    return average_volume_test

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

In [7]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-10-20_1:5:41 : , 0 , 000020 , 동화약품 -------------------  
X_1. ratio_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-10-20_1:5:41 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-10-20_1:5:41 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-10-20_1:5:41 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-10-20_1:5:41 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-10-20_1:5:42 : , 5 , 000075 , 삼양홀딩스우 -------------------  
X_1. ratio_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-10-20_1:5:42 : , 6 , 000080 , 하이트진로 -------------------  
X_1. ratio_test 없음 6 , 000080 , 하이트진로
------------------- test 시작_2019-10-20_1:5:42 : , 7 , 000087 , 하이트진로2우B -------------------  
X_1. rati

O_1. ratio_test 통과 57 , 000910 , 유니온
O_2. total_volume_test 통과 57 , 000910 , 유니온
X_3. average_volume_test 없음 57 , 000910 , 유니온
------------------- test 시작_2019-10-20_1:5:52 : , 58 , 000950 , 전방 -------------------  
X_1. ratio_test 없음 58 , 000950 , 전방
------------------- test 시작_2019-10-20_1:5:52 : , 59 , 000970 , 한국주철관 -------------------  
X_1. ratio_test 없음 59 , 000970 , 한국주철관
------------------- test 시작_2019-10-20_1:5:52 : , 60 , 000990 , DB하이텍 -------------------  
X_1. ratio_test 없음 60 , 000990 , DB하이텍
------------------- test 시작_2019-10-20_1:5:53 : , 61 , 000995 , DB하이텍1우 -------------------  
O_1. ratio_test 통과 61 , 000995 , DB하이텍1우
X_2. total_volume_test 없음 61 , 000995 , DB하이텍1우
------------------- test 시작_2019-10-20_1:5:53 : , 62 , 001020 , 페이퍼코리아 -------------------  
O_1. ratio_test 통과 62 , 001020 , 페이퍼코리아
X_2. total_volume_test 없음 62 , 001020 , 페이퍼코리아
------------------- test 시작_2019-10-20_1:5:53 : , 63 , 001040 , CJ -------------------  
X_1. ratio_test 없음 63 , 001040 , C

------------------- test 완료 : , 111 , 001745 , SK네트웍스우 ------------------- 
------------------- test 시작_2019-10-20_1:6:3 : , 112 , 001750 , 한양증권 -------------------  
X_1. ratio_test 없음 112 , 001750 , 한양증권
------------------- test 시작_2019-10-20_1:6:3 : , 113 , 001755 , 한양증권우 -------------------  
X_1. ratio_test 없음 113 , 001755 , 한양증권우
------------------- test 시작_2019-10-20_1:6:3 : , 114 , 001770 , 신화실업 -------------------  
O_1. ratio_test 통과 114 , 001770 , 신화실업
X_2. total_volume_test 없음 114 , 001770 , 신화실업
------------------- test 시작_2019-10-20_1:6:3 : , 115 , 001780 , 알루코 -------------------  
X_1. ratio_test 없음 115 , 001780 , 알루코
------------------- test 시작_2019-10-20_1:6:3 : , 116 , 001790 , 대한제당 -------------------  
X_1. ratio_test 없음 116 , 001790 , 대한제당
------------------- test 시작_2019-10-20_1:6:4 : , 117 , 001795 , 대한제당우 -------------------  
O_1. ratio_test 통과 117 , 001795 , 대한제당우
X_2. total_volume_test 없음 117 , 001795 , 대한제당우
------------------- test 시작_2019-10-20_1:6:4 : , 

O_1. ratio_test 통과 158 , 002787 , 진흥기업2우B
X_2. total_volume_test 없음 158 , 002787 , 진흥기업2우B
------------------- test 시작_2019-10-20_1:6:12 : , 159 , 002790 , 아모레G -------------------  
X_1. ratio_test 없음 159 , 002790 , 아모레G
------------------- test 시작_2019-10-20_1:6:12 : , 160 , 002795 , 아모레G우 -------------------  
X_1. ratio_test 없음 160 , 002795 , 아모레G우
------------------- test 시작_2019-10-20_1:6:12 : , 161 , 002810 , 삼영무역 -------------------  
X_1. ratio_test 없음 161 , 002810 , 삼영무역
------------------- test 시작_2019-10-20_1:6:13 : , 162 , 002820 , 선창산업 -------------------  
X_1. ratio_test 없음 162 , 002820 , 선창산업
------------------- test 시작_2019-10-20_1:6:13 : , 163 , 002840 , 미원상사 -------------------  
O_1. ratio_test 통과 163 , 002840 , 미원상사
X_2. total_volume_test 없음 163 , 002840 , 미원상사
------------------- test 시작_2019-10-20_1:6:13 : , 164 , 002870 , 신풍제지 -------------------  
X_1. ratio_test 없음 164 , 002870 , 신풍제지
------------------- test 시작_2019-10-20_1:6:13 : , 165 , 002880 , 대유에이텍 ----

O_1. ratio_test 통과 212 , 003680 , 한성기업
X_2. total_volume_test 없음 212 , 003680 , 한성기업
------------------- test 시작_2019-10-20_1:6:23 : , 213 , 003690 , 코리안리 -------------------  
X_1. ratio_test 없음 213 , 003690 , 코리안리
------------------- test 시작_2019-10-20_1:6:23 : , 214 , 003720 , 삼영화학 -------------------  
O_1. ratio_test 통과 214 , 003720 , 삼영화학
X_2. total_volume_test 없음 214 , 003720 , 삼영화학
------------------- test 시작_2019-10-20_1:6:23 : , 215 , 003780 , 진양산업 -------------------  
X_1. ratio_test 없음 215 , 003780 , 진양산업
------------------- test 시작_2019-10-20_1:6:23 : , 216 , 003830 , 대한화섬 -------------------  
X_1. ratio_test 없음 216 , 003830 , 대한화섬
------------------- test 시작_2019-10-20_1:6:23 : , 217 , 003850 , 보령제약 -------------------  
O_1. ratio_test 통과 217 , 003850 , 보령제약
O_2. total_volume_test 통과 217 , 003850 , 보령제약
O_3. average_volume_test 통과 217 , 003850 , 보령제약
------------------- test 완료 : , 217 , 003850 , 보령제약 ------------------- 
------------------- test 시작_2019-10-20_1:6:24 :

...

X_1. ratio_test 없음 2704 , 226440 , 한송네오텍
------------------- test 시작_2019-10-20_1:14:33 : , 2705 , 226950 , 올릭스 -------------------  
O_1. ratio_test 통과 2705 , 226950 , 올릭스
O_2. total_volume_test 통과 2705 , 226950 , 올릭스
X_3. average_volume_test 없음 2705 , 226950 , 올릭스
------------------- test 시작_2019-10-20_1:14:34 : , 2706 , 227100 , 디자인 -------------------  
X_1. ratio_test 없음 2706 , 227100 , 디자인
------------------- test 시작_2019-10-20_1:14:34 : , 2707 , 227610 , 아우딘퓨쳐스 -------------------  
O_1. ratio_test 통과 2707 , 227610 , 아우딘퓨쳐스
X_2. total_volume_test 없음 2707 , 227610 , 아우딘퓨쳐스
------------------- test 시작_2019-10-20_1:14:34 : , 2708 , 227950 , 마이크로텍 -------------------  
O_1. ratio_test 통과 2708 , 227950 , 마이크로텍
X_2. total_volume_test 없음 2708 , 227950 , 마이크로텍
------------------- test 시작_2019-10-20_1:14:34 : , 2709 , 228340 , 동양파일 -------------------  
X_1. ratio_test 없음 2709 , 228340 , 동양파일
------------------- test 시작_2019-10-20_1:14:34 : , 2710 , 228670 , 레이 -------------------  
O_1.

------------------- test 완료 : , 2750 , 250060 , 모비스 ------------------- 
------------------- test 시작_2019-10-20_1:14:42 : , 2751 , 250930 , 예선테크 -------------------  
O_1. ratio_test 통과 2751 , 250930 , 예선테크
X_2. total_volume_test 없음 2751 , 250930 , 예선테크
------------------- test 시작_2019-10-20_1:14:43 : , 2752 , 251370 , 와이엠티 -------------------  
X_1. ratio_test 없음 2752 , 251370 , 와이엠티
------------------- test 시작_2019-10-20_1:14:43 : , 2753 , 251630 , 브이원텍 -------------------  
O_1. ratio_test 통과 2753 , 251630 , 브이원텍
X_2. total_volume_test 없음 2753 , 251630 , 브이원텍
------------------- test 시작_2019-10-20_1:14:43 : , 2754 , 251970 , 펌텍코리아 -------------------  
O_1. ratio_test 통과 2754 , 251970 , 펌텍코리아
O_2. total_volume_test 통과 2754 , 251970 , 펌텍코리아
X_3. average_volume_test 없음 2754 , 251970 , 펌텍코리아
------------------- test 시작_2019-10-20_1:14:43 : , 2755 , 252500 , 세화피앤씨 -------------------  
O_1. ratio_test 통과 2755 , 252500 , 세화피앤씨
O_2. total_volume_test 통과 2755 , 252500 , 세화피앤씨
X_3. average_

X_1. ratio_test 없음 2796 , 265920 , 한화수성스팩
------------------- test 시작_2019-10-20_1:14:52 : , 2797 , 267320 , 교보7호스팩 -------------------  
X_1. ratio_test 없음 2797 , 267320 , 교보7호스팩
------------------- test 시작_2019-10-20_1:14:52 : , 2798 , 267790 , 배럴 -------------------  
X_1. ratio_test 없음 2798 , 267790 , 배럴
------------------- test 시작_2019-10-20_1:14:52 : , 2799 , 267980 , 매일유업 -------------------  
X_1. ratio_test 없음 2799 , 267980 , 매일유업
------------------- test 시작_2019-10-20_1:14:52 : , 2800 , 268600 , 셀리버리 -------------------  
O_1. ratio_test 통과 2800 , 268600 , 셀리버리
O_2. total_volume_test 통과 2800 , 268600 , 셀리버리
X_3. average_volume_test 없음 2800 , 268600 , 셀리버리
------------------- test 시작_2019-10-20_1:14:52 : , 2801 , 269620 , 시스웍 -------------------  
O_1. ratio_test 통과 2801 , 269620 , 시스웍
O_2. total_volume_test 통과 2801 , 269620 , 시스웍
O_3. average_volume_test 통과 2801 , 269620 , 시스웍
------------------- test 완료 : , 2801 , 269620 , 시스웍 ------------------- 
------------------- test 시작

X_1. ratio_test 없음 2844 , 302430 , 이노메트리
------------------- test 시작_2019-10-20_1:15:1 : , 2845 , 303030 , 지니틱스 -------------------  
O_1. ratio_test 통과 2845 , 303030 , 지니틱스
O_2. total_volume_test 통과 2845 , 303030 , 지니틱스
O_3. average_volume_test 통과 2845 , 303030 , 지니틱스
------------------- test 완료 : , 2845 , 303030 , 지니틱스 ------------------- 
------------------- test 시작_2019-10-20_1:15:1 : , 2846 , 305090 , 마이크로디지탈 -------------------  
X_1. ratio_test 없음 2846 , 305090 , 마이크로디지탈
------------------- test 시작_2019-10-20_1:15:2 : , 2847 , 306620 , DB금융스팩6호 -------------------  
X_1. ratio_test 없음 2847 , 306620 , DB금융스팩6호
------------------- test 시작_2019-10-20_1:15:2 : , 2848 , 307070 , SK4호스팩 -------------------  
X_1. ratio_test 없음 2848 , 307070 , SK4호스팩
------------------- test 시작_2019-10-20_1:15:2 : , 2849 , 307160 , 하나머스트제6호스팩 -------------------  
X_1. ratio_test 없음 2849 , 307160 , 하나머스트제6호스팩
------------------- test 시작_2019-10-20_1:15:2 : , 2850 , 307180 , 신영스팩4호 -------------------  

O_1. ratio_test 통과 2891 , 900250 , 크리스탈신소재
O_2. total_volume_test 통과 2891 , 900250 , 크리스탈신소재
O_3. average_volume_test 통과 2891 , 900250 , 크리스탈신소재
------------------- test 완료 : , 2891 , 900250 , 크리스탈신소재 ------------------- 
------------------- test 시작_2019-10-20_1:15:10 : , 2892 , 900260 , 로스웰 -------------------  
O_1. ratio_test 통과 2892 , 900260 , 로스웰
O_2. total_volume_test 통과 2892 , 900260 , 로스웰
O_3. average_volume_test 통과 2892 , 900260 , 로스웰
------------------- test 완료 : , 2892 , 900260 , 로스웰 ------------------- 
------------------- test 시작_2019-10-20_1:15:11 : , 2893 , 900270 , 헝셩그룹 -------------------  
O_1. ratio_test 통과 2893 , 900270 , 헝셩그룹
O_2. total_volume_test 통과 2893 , 900270 , 헝셩그룹
O_3. average_volume_test 통과 2893 , 900270 , 헝셩그룹
------------------- test 완료 : , 2893 , 900270 , 헝셩그룹 ------------------- 
------------------- test 시작_2019-10-20_1:15:11 : , 2894 , 900280 , 골든센츄리 -------------------  
X_1. ratio_test 없음 2894 , 900280 , 골든센츄리
------------------- test 시작_2019-10-20_